In [1]:
from IPython.display import display,Image,clear_output
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import seaborn as sns
import random
import urllib.request
import h5py

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import get_file
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, InceptionV3
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.models import save_model

In [4]:
first_gate = VGG16(weights='imagenet')

In [8]:
from keras.models import load_model
second_gate = load_model(r'E:\code\Car_damage_evaluation\car-damage-dataset\data1a\ft_model.keras')
location_model = load_model(r'E:\code\Car_damage_evaluation\car-damage-dataset\data2a\ft_model.keras')
severity_model = load_model(r'E:\code\Car_damage_evaluation\car-damage-dataset\data3a\ft_model.keras')

In [9]:
import os
print(os.path.exists(r'E:\code\Car_damage_evaluation\car-damage-dataset\data1a\ft_model.keras'))


True


In [10]:
with open('vgg16_cat_list.pk','rb') as f:
    cat_list = pickle.load(f)

In [11]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04461696', 'tow_truck'),
 ('n03594945', 'jeep'),
 ('n04037443', 'racer'),
 ('n02974003', 'car_wheel'),
 ('n03459775', 'grille'),
 ('n04252225', 'snowplow'),
 ('n03769881', 'minibus'),
 ('n02930766', 'cab'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n03445924', 'golfcart'),
 ('n03417042', 'garbage_truck'),
 ('n02701002', 'ambulance'),
 ('n03791053', 'motor_scooter'),
 ('n03478589', 'half_track'),
 ('n02704792', 'amphibian'),
 ('n04252077', 'snowmobile'),
 ('n04467665', 'trailer_truck'),
 ('n03777568', 'Model_T'),
 ('n02965783', 'car_mirror'),
 ('n02917067', 'bullet_train'),
 ('n04065272', 'recreational_vehicle'),
 ('n03345487', 'fire_engine'),
 ('n04465501', 'tractor'),
 ('n03127747', 'crash_helmet'),
 ('n03891332', 'parking_meter'),
 ('n02747177', 'ashcan'),
 ('n04389033', 'tank'),
 ('n0364

In [12]:
# from Keras GitHub  
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'

def get_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [13]:
def prepare_img_244(img_path):
    #urllib.request.urlretrieve(img_url,'save.jpg')
    img = load_img(img_path,target_size=(224,224))
    x = img_to_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    return x

In [14]:
def car_categories_gate(img244,model):
    print("Validating that this is a picture of your car...")
    out = model.predict(img244)
    top = get_predictions(out,top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            # print j[0:2]
            return True # "Validation complete - proceed to damage evaluation"
    return False # "Are you sure this is a picture of your car? 
                 # Please take another picture (try a different angle or lighting) and try again."

In [15]:
def prepare_img_256(img_path):
    img = load_img(img_path,target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255  
    return x

In [16]:
def car_damage_gate(img_256,model):
    print("Validating that damage exists...")
    pred = model.predict(img_256)
    if pred[0][0] <=.5:
        return True # print "Validation complete - proceed to location and severity determination"
    else:
        return False
        # print "Are you sure that your car is damaged? Please submit another picture of the damage."
        # print "Hint: Try zooming in/out, using a different angle or different lighting"

In [17]:
def location_assessment(img_256,model):
    print("Determining location of damage...")
    pred = model.predict(img_256)
    pred_label = np.argmax(pred,axis=1)
    d = {0: 'Front', 1: 'Rear', 2: 'Side'}
    for key in d.iterkeys():
        if pred_label[0] == key:
            print("Assessment: {} damage to vehicle".format(d[key]))
    print("Location assessment complete.")

In [18]:
def severity_assessment(img_256, model):
    print("Determining severity of damage...")
    pred = model.predict(img_256)
    pred_label = np.argmax(pred, axis=1)
    d = {0: 'Minor', 1: 'Moderate', 2: 'Severe'}
    for key in d.iterkeys():
        if pred_label[0] == key:
            print("Assessment: {} damage to vehicle".format(d[key]))
    print("Severity assessment complete.")

In [20]:
# load models
def engine(img_path):
    img_244 = prepare_img_244(img_path)
    g1 = car_categories_gate(img_244,first_gate)
    if g1 is False:
        result = {'gate1': 'Car validation check: ', 
        'gate1_result': 0, 
        'gate1_message': {0: 'Are you sure this is a picture of your car? Please retry your submission.', 
        1: 'Hint: Try zooming in/out, using a different angle or different lighting'},
        'gate2': None,
        'gate2_result': None,
        'gate2_message': {0: None, 1: None},
        'location': None,
        'severity': None,
        'final': 'Damage assessment unsuccessful!'}
        return result

    img_256 = prepare_img_256(img_path)
    g2 = car_damage_gate(img_256,second_gate)
    if g2 is False:
        result = {'gate1': 'Car validation check: ', 
        'gate1_result': 1, 
        'gate1_message': {0: None, 1: None},
        'gate2': 'Damage presence check: ',
        'gate2_result': 0,
        'gate2_message': {0: 'Are you sure that your car is damaged? Please retry your submission.',
        1: 'Hint: Try zooming in/out, using a different angle or different lighting.'},
        'location': None,
        'severity': None,
        'final': 'Damage assessment unsuccessful!'}
        return result

    x = location_assessment(img_256,location_model)
    y = severity_assessment(img_256,severity_model)

    result = {'gate1': 'Car validation check: ', 
	'gate1_result': 1, 
	'gate1_message': {0: None, 1: None},
	'gate2': 'Damage presence check: ',
	'gate2_result': 1,
	'gate2_message': {0: None, 1: None},
	'location': x,
	'severity': y,
	'final': 'Damage assessment complete!'}
    return result